Fully Connected Network


In [1]:
#Necessary Imports
import tensorflow as tf
import numpy as np
import os 
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

c:\Users\rithv\GitHub\brain-mri-scans\code\venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


Define dataset path and initialize containers

<div style="font-size:85%">
 
1. Set folder_path to the directory containing image patches.  
2. Initialize lists to collect image arrays and labels.
</div>

In [2]:
folder_path = "C:/Users/rithv/GitHub/brain-mri-scans/code/sub_dataset/patches/"
images = []
labels = []

Load and preprocess images

<div style="font-size:85%">
  
1. Iterate sorted filenames in folder_path.  
2. For each PNG:  
   1. Read in grayscale with cv2.  
   2. Resize to 64×64 pixels.  
   3. Normalize pixel values to [0, 1].  
   4. Append the processed image to images list.  

</div>

In [3]:
for filename in sorted(os.listdir(folder_path)):
    if filename.endswith(".png"):
        img = cv2.imread(os.path.join(folder_path, filename), cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (64, 64))
        
        img = img / 255.0

        images.append(img)

Load labels and prepare arrays

<div style="font-size:85%">

1. Convert images list to NumPy array x.
2. Load labels from CSV (column index 3), skipping header.
3. Print labels for quick inspection.
4. Convert labels to NumPy array y.
</div>

In [4]:
x = np.array(images)
labels = np.genfromtxt(r"C:\Users\rithv\GitHub\brain-mri-scans\code\sub_dataset\sub_dataset.csv", delimiter=",", skip_header=1, usecols=3)
print(labels)
y = np.array(labels)

[0. 0. 0. ... 0. 0. 0.]


Initial train/test split

<div style="font-size:85%">

1. Creates an 80/20 stratified split of x and y.
</div>

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

Define the training function for a fully connected model

<div style="font-size:85%">

1. Convert inputs x and y to float32 NumPy arrays.
2. Perform a stratified train/test split.
3. Build a Sequential model:
   1. Flatten input.
   2. Dense(128, relu) → Dense(64, relu) → Dense(1, sigmoid).
4. Compile with Adam optimizer and binary_crossentropy loss; track accuracy.
5. Train with validation_split and return model, history, and test metrics after evaluation.
</div>

In [ ]:
def fully_connected(x, y, input_shape=(64, 64), epochs=5, batch_size=16, test_size=0.2, random_state=42):
    x = np.asarray(x, dtype=np.float32)
    y = np.asarray(y, dtype=np.float32)

    x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=test_size, random_state=random_state, stratify=y
    )

    model = Sequential([
        Flatten(input_shape=input_shape),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    history = model.fit(
        x_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.2,
        verbose=1
    )

    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    print("Test Accuracy:", accuracy)
    return model, history, (loss, accuracy)

Run the model and print test accuracy

<div style="font-size:85%">

1. Call run_fully_connected with chosen epochs and batch size.
2. Receive the trained model, training history, and test metrics.
3. Test accuracy is printed by the function and stored in test_metrics.
</div>

In [ ]:

model, history, test_metrics = fully_connected(x, y, epochs=5, batch_size=16)


c:\Users\rithv\GitHub\brain-mri-scans\code\venv\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
192/192 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.9004 - loss: 0.3441 - val_accuracy: 0.8958 - val_loss: 0.2724
Epoch 2/5
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9043 - loss: 0.2664 - val_accuracy: 0.9076 - val_loss: 0.2443
Epoch 3/5
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9089 - loss: 0.2415 - val_accuracy: 0.9062 - val_loss: 0.2283
Epoch 4/5
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9108 - loss: 0.2370 - val_accuracy: 0.9167 - val_loss: 0.2223
Epoch 5/5
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9150 - loss: 0.2222 - val_accuracy: 0.9180 - val_loss: 0.2222
Test Accuracy: 0.9166666865348816
